<a href="https://colab.research.google.com/github/jhay20ng/bigdata-ai/blob/main/bigkinds_%EB%89%B4%EC%8A%A4_%EA%B8%B0%EC%82%AC_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm_notebook
import pandas as pd

from selenium.webdriver.common.keys import Keys
BACKSPACE='/ue003'
ENTER='/ue007'
TAB='/ue004'

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import shutil

try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제
except FileNotFoundError:
    pass

subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동


option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
    
driver.implicitly_wait(10)

# 구글 드라이브 실행후 '빅카인즈' 사이트에 들어감
driver.get('https://www.bigkinds.or.kr/v2/news/index.do')

In [ ]:
titles = ['불황']
start_date = ['2010-06-16']
end_date= ['2010-12-31']

for i in titles:
    driver.find_element_by_xpath('//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[3]/a').click()
    elem = driver.find_element_by_id ('total-search-key')
    elem.send_keys(i)
    
    time.sleep(1)
    
    elem.send_keys(Keys.TAB)
    
    time.sleep(1)
    
    # 언론사 '경상' 클릭()
    driver.find_element_by_xpath('//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[3]/a').click()
    driver.find_element_by_xpath('//*[@id="category_provider_area"]/span[5]/label').click()
    
    time.sleep(1)
    
    # 기간 설정
    # '기간'이라는 창 클릭
    driver.find_element_by_xpath('//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
    
    time.sleep(1)
    
    # 기간 입력
    for start, end in zip(start_date, end_date):
        elem1 = driver.find_element_by_id ('search-begin-date')
        for i in range(10):
            elem1.send_keys(Keys.BACKSPACE)
        elem1.send_keys(start)
    
        elem2 = driver.find_element_by_id ('search-end-date')
        for i in range(10):
            elem2.send_keys(Keys.BACKSPACE)
        elem2.send_keys(end) 
        
        time.sleep(1)
        
        # 통합 분류창 클릭
        driver.find_element_by_xpath('//*[@id="collapse-step-1-body"]/div[3]/div/div[2]/div[1]/a').click()

        time.sleep(1)

        # '경제' 클릭
        driver.find_element_by_xpath('//*[@id="srch-tab3"]/ul/li[2]/div/span[3]/label/span').click()

        # '적용하기'로 검색
        driver.find_element_by_xpath('//*[@id="search-foot-div"]/div[2]/button[2]').click()
        
        time.sleep(1)
        
        # 100건씩 보기 클릭
        driver.find_element_by_xpath('//*[@id="select2"]/option[4]').click()
    
        time.sleep(2)
        
        #### 뉴스기사 크롤링 시작
        title = []
        content = []
        number = 0
        
        # 검색한 키워드의 뉴스 페이지수
        pages = driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[1]/div[2]/div/div/div/div/div[3]/div/b')
        page = int(pages.text)
        
        # 뉴스 총 검색 건수
        a = driver.find_element_by_xpath('/html/body/div[1]/main/div[1]/div[2]/div/div[2]/div[2]/div/div[2]/div[3]/div[2]/h3/span[6]')
        a = a.text
        total_number = int(a.replace(",",''))
        remain_number = total_number%100
        present_number = 1
        
        time.sleep(1)
        
        while number <= page:
            # 현재 페이지 숫자
            numbers = driver.find_element_by_id("paging_news_result").get_attribute("value")
            number = int(numbers)
                                                                                                                                    
            # 각 페이지에서 기사 100개 크롤링 코드
            for i in range(1,101):
                
                time.sleep(1)
                
                # 기사 본문 내용 들어있는 창 클릭하기
                driver.find_element_by_xpath('//*[@id="news-results"]/div[%d]/div/div[2]/a/div/strong/span'%(i)).click()
                
                time.sleep(4)
                
                # 기사 제목과 기사 내용 가져오기
                news_title = driver.find_elements_by_css_selector("#news-detail-modal > div > div > div.modal-body > div > div.news-view-head > h1")
                news_text = driver.find_elements_by_css_selector("#news-detail-modal > div > div > div.modal-body > div > div.news-view-body")
                
                time.sleep(1)
                
                # 가져온 내용들 텍스트형태로 변환
                for titles, contents in zip(news_title, news_text):
                    time.sleep(2)
                    titles1 = titles.text
                    contents1 = contents.text
                    title.append(titles1)
                    content.append(contents1)
                
                time.sleep(1)
                
                # 기사 본문 내용 띄워진 창 닫기
                driver.find_element_by_xpath('/html/body/div[4]/div/div/button').click()
        
                if (number == page) and (i==remain_number):
                    break
    
            # 다음 페이지 넘어가기
            driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[2]/div/div/div/div/div[4]/a').click()
        
            time.sleep(2)
    
            if number == page:
                break
                    
        # 파일명을 자동 변경해주는 코드 
        input_path = r'C:/Users/Bigdata/파이썬프로그래밍실무/키워드/original/'
        filename='bigkinds' #파일명 고정값
        file_ext='.csv' #파일 형식
         
        output_path='C:/Users/Bigdata/파이썬프로그래밍실무/키워드/%s%s' %(filename,file_ext)
        uniq=1
        while os.path.exists(output_path):  #동일한 파일명이 존재할 때
            output_path='C:/Users/Bigdata/파이썬프로그래밍실무/키워드/%s(%d)%s' % (filename,uniq,file_ext) #파일명(1) 파일명(2)...
            uniq+=1
        
        
        # 데이터프레임 csv파일 형태로 저장
        news_df = pd.DataFrame({'title': title, 'content': content})
        news_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        time.sleep(1)
        
        # 내려가있는 화면 'Top'버튼 클릭하여 화면 올리기
        driver.find_element_by_xpath('//*[@id="wrap"]/button').click()
        
        time.sleep(1)
        
        # STEP 01 - 뉴스 검색 창 클릭
        driver.find_element_by_xpath('//*[@id="collapse-step-1"]').click()
                
        time.sleep(1)
        
        # '경제' 클릭
        driver.find_element_by_xpath('//*[@id="srch-tab3"]/ul/li[2]/div/span[3]/label/span').click()
        
        time.sleep(1)
        
        # '기간'이라는 창 클릭
        driver.find_element_by_xpath('//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
        
    # 뉴스 검색/분석 초기창 클릭
    driver.find_element_by_xpath('//*[@id="header"]/div[1]/div/div[1]/div[2]/div/div/div[2]/div/div[1]/ul/li[1]/a').click()
       
    time.sleep(1)

In [ ]:
# 데이터프레임 csv파일 형태로 저장
news_df = pd.DataFrame({'title': title, 'content': content})
news_df.to_csv(output_path, index=False, encoding='utf-8-sig')